# Ch05 금융 지점 데이터 분석 통합

이 노트북은 Ch05 폴더의 모든 분석 코드를 통합한 것입니다.
- 데이터 로드 및 기본 정보 확인
- 업종별/구별 점포 수 분석
- 시간에 따른 변화 분석
- 특정 지역 상세 분석

In [2]:
# 기본 라이브러리 및 데이터 로드
import pandas as pd
import seaborn as sns
import folium
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 부호 깨짐 방지

ym = ["201512", "201606", "201612", "201706", "201712"]

# 금융 지점 데이터 읽기 (이미 통합 csv)
df = pd.read_csv((Path.cwd() / 'data' / 'finance_branches.csv'))
df.head()

수집연월 시군구명  행정동명 상권업종대분류명 상권업종중분류명 상권업종소분류명  상가업소번호       위도        경도
0     0  강남구  역삼1동       금융       은행       지점    1001  37.4996  127.0364
1     0  강남구  역삼1동       증권     증권중개    브로커리지    1002  37.5001  127.0340
2     0  서초구  서초1동       보험     손해보험      영업소    1003  37.4924  127.0079
3     0  송파구  잠실본동       금융       은행       지점    1004  37.5053  127.0850
4     0  마포구   합정동       증권     투자은행      영업소    1005  37.5499  126.9140

In [ ]:
# 데이터 기본 정보 확인
df.info()

In [ ]:
df['수집연월'].unique()

In [ ]:
df.groupby('상권업종대분류명')['상권업종대분류명'].count()

In [ ]:
df.groupby('상권업종중분류명')['상권업종중분류명'].unique().index

In [ ]:
df.groupby('상권업종소분류명')['상권업종소분류명'].unique().index

In [ ]:
# 결측값 확인
df.isna().sum()

In [ ]:
# 201712 수집 데이터만 추출
df_201712 = df.loc[df.수집연월 == 4, :]
df_201712.shape

In [ ]:
# 업종별 점포 수 그래프 그리기
result = df_201712.groupby('상권업종대분류명')['상가업소번호'].count()
result

In [ ]:
plt.figure(figsize=(10, 6))
fig = sns.barplot(data=result)
fig.set_ylabel('점포수')
fig.set_title('업종별 점포수')
plt.show()

In [ ]:
# 구별 점포 수 그래프 그리기
result = df_201712.groupby('시군구명')['시군구명'].count()
result

In [ ]:
plt.figure(figsize=(10, 6))
fig = sns.barplot(data=result)
fig.set_ylabel('점포수')
fig.set_xlabel('')
fig.set_title('구별 점포수')
fig.set_xticklabels(fig.get_xticklabels(), rotation=45)
plt.show()

In [ ]:
# 지도의 중심점 구하기
center = [df_201712['위도'].mean(), df_201712['경도'].mean()]
center

In [ ]:
# 구의 중심점 구하기
loc = df_201712.groupby('시군구명')[['위도', '경도']].mean()
loc

In [ ]:
# 구의 점포 수 구하기
store = df_201712.groupby('시군구명')['시군구명'].count()
store

In [ ]:
df_store = pd.concat([store, loc], axis=1)
df_store.columns = ['점포수', '위도', '경도']
df_store

In [ ]:
# 점포 수를 원의 크기로 표시하기
_map = folium.Map(location=center, zoom_start=12)  # 지도 가져오기

# 지도 위에 점포 수를 원의 크기로 표시하기
for i in range(len(df_store)):
    folium.CircleMarker(location=[df_store['위도'].iloc[i],
                                  df_store['경도'].iloc[i]],
                        radius=(df_store['점포수'].iloc[i])*0.5 + 5,  # 원의 반지름
                        color='red',  # 원의 색
                        stroke=False,  # 윤곽선 X
                        fill=True,  # 원의 내부 색
                        fill_opacity=0.3  # 원의 투명도
                        ).add_to(_map)

# 지도에 구 이름 추가하기
for i in range(len(df_store)):
    folium.Marker(location=[df_store['위도'].iloc[i],
                            df_store['경도'].iloc[i]],
                  icon=folium.DivIcon(
        icon_anchor=(0, 0),  # 텍스트 위치 설정
        html=f"<div style='font-size:12px;color:blue;background-color:rgba(255,255,255,0.2);width:85px;text-align:left;margin:0px;'><b>{df_store.index[i]}</b></div>"
    )).add_to(_map)

_map.save('finance_store_map.html')
print('지도가 finance_store_map.html 파일로 저장되었습니다.')

In [ ]:
# 점포 수가 많은 상위 10개 동 알아보기
store = df_201712.groupby('행정동명')['행정동명'].count()
store = store.sort_values(ascending=False).head(10)
store

In [ ]:
plt.figure(figsize=(10, 6))
fig = sns.barplot(data=store)
fig.set_ylabel('점포수')
fig.set_xlabel('')
fig.set_title('점포수 많은 상위 10개동')
fig.set_xticklabels(fig.get_xticklabels(), rotation=45)
plt.subplots_adjust(bottom=0.2)
plt.show()

In [ ]:
# 업종별 (대분류) 점포 수 변화
store_change = df.groupby(['수집연월', '상권업종대분류명'])['행정동명'].count()
store_change = store_change.reset_index()
store_change.columns = ['수집연월', '상권업종대분류명', '점포수']
store_change

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_theme(style='darkgrid', font='Malgun Gothic')
fig = sns.lineplot(x='수집연월', y='점포수',
                   hue='상권업종대분류명', marker='o',
                   data=store_change)

fig.set_xticks(range(5))
fig.set_xticklabels(ym)
fig.set_title('업종별 점포수 변화 (대분류)')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.subplots_adjust(right=0.75)
plt.show()

In [ ]:
# 점포 수 변화가 큰 상위 업종 (소분류)
store_change = pd.pivot_table(df,
                              index='상권업종소분류명',
                              columns='수집연월',
                              values='상가업소번호',
                              aggfunc='count'
                              ).fillna(0)
if 4 in store_change.columns and 0 in store_change.columns:
    store_change['diff'] = abs(store_change[4]-store_change[0])
else:
    store_change['diff'] = 0
store_change = store_change.sort_values('diff', ascending=False).head(10)
store_change  # 상위 10개 업종

In [ ]:
top10 = store_change.reset_index().rename_axis(None, axis=1)
top10 = pd.melt(top10, id_vars=['상권업종소분류명'], value_vars=[0, 1, 2, 3, 4])
top10.columns = ['상권업종소분류명', '수집연월', '점포수']
top10

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_theme(style='darkgrid', font='Malgun Gothic')
fig = sns.lineplot(x='수집연월', y='점포수',
                   hue='상권업종소분류명', marker='o',
                   data=top10)

fig.set_xticks(range(5))
fig.set_xticklabels(ym)
fig.set_title('업종별 점포수 변화 (소분류)')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.subplots_adjust(right=0.75)
plt.show()

In [ ]:
# 구별 점포 수의 변화
store_change = df.groupby(['시군구명', '수집연월'])['상가업소번호'].count()
store_change = store_change.reset_index().rename_axis(None, axis=1)
store_change.columns = ['시군구명', '수집연월', '점포수']
store_change

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_theme(style='darkgrid', font='Malgun Gothic')
fig = sns.lineplot(x='수집연월', y='점포수',
                   hue='시군구명', marker='o',
                   data=store_change)

fig.set_xticks(range(5))
fig.set_xticklabels(ym)
fig.set_title('구별 점포수 변화')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, ncols=2, borderaxespad=0)
plt.subplots_adjust(right=0.7)
plt.show()

In [ ]:
# 점포 수 변화가 큰 상위 동 확인
store_change = pd.pivot_table(df,
                              index='행정동명',
                              columns='수집연월',
                              values='상가업소번호',
                              aggfunc='count'
                              ).fillna(0)
if 4 in store_change.columns and 0 in store_change.columns:
    store_change['diff'] = abs(store_change[4]-store_change[0])
else:
    store_change['diff'] = 0
store_change = store_change.sort_values('diff', ascending=False).head(10)
store_change

In [ ]:
top10 = store_change.reset_index().rename_axis(None, axis=1)
top10 = pd.melt(top10, id_vars=['행정동명'], value_vars=[0, 1, 2, 3, 4])
top10.columns = ['행정동명', '수집연월', '점포수']
top10

In [ ]:
plt.figure(figsize=(10, 6))
sns.set_theme(style='darkgrid', font='Malgun Gothic')
fig = sns.lineplot(x='수집연월', y='점포수',
                   hue='행정동명', marker='o',
                   data=top10)

fig.set_xticks(range(5))
fig.set_xticklabels(ym)
fig.set_title('점포수 변화가 많은 동')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.subplots_adjust(right=0.75)
plt.show()

In [ ]:
# 역삼1동 금융 점포 분석
df_sel = df.loc[(df.행정동명 == '역삼1동') & (df.수집연월 == 4), :]

dict_color = {'금융': 'yellow', '증권': 'red',
              '보험': 'green'}

#점포를 업종별(대분류)로 구분하여 지도에 표시하기
center = [df_sel['위도'].mean(), df_sel['경도'].mean()]

_map = folium.Map(location=center, zoom_start=15)  # 지도 가져오기

for i in range(len(df_sel)):
    folium.CircleMarker(location=[df_sel['위도'].iloc[i],
                                  df_sel['경도'].iloc[i]],
                                  radius=5,  # 원의 반지름
                                  color=dict_color.get(df_sel['상권업종대분류명'].iloc[i], 'blue'),  # 원의 색
                                  stroke=False,  # 윤곽선 X
                                  fill=True,  # 원의 색
                                  fill_opacity=0.9  # 원의 투명도
                                  ).add_to(_map)

_map.save('finance_branch_map_yeoksam.html')
print('역삼1동 지도가 finance_branch_map_yeoksam.html 파일로 저장되었습니다.')

In [ ]:
# 은행 지점만 지도에 표시하기
df_bank = df.loc[(df.행정동명 == '역삼1동') & (df.수집연월 == 4) &
                (df.상권업종소분류명 == '지점'), :]

center = [df_bank['위도'].mean(), df_bank['경도'].mean()]

_map2 = folium.Map(location=center, zoom_start=15)  # 지도 가져오기

for i in range(len(df_bank)):
    folium.CircleMarker(location=[df_bank['위도'].iloc[i],
                                df_bank['경도'].iloc[i]],
                        radius=6,  # 원의 반지름
                        color='red',  # 원의 색
                        stroke=False,  # 윤곽선 X
                        fill=True,  # 원의 내부 색
                        fill_opacity=0.9  # 원의 투명도
                        ).add_to(_map2)

_map2.save('finance_bank_only_yeoksam.html')
print('역삼1동 은행 지점만 표시한 지도가 finance_bank_only_yeoksam.html 파일로 저장되었습니다.')